In [1]:
import pandas as pd 
file_path = "data/raw/catusita/Data de venta 01.01.21 a 06.12.24.xls"
df_catusita = pd.read_excel(file_path, sheet_name="Sheet1")
excel_file = pd.ExcelFile(file_path)
list_hojas = excel_file.sheet_names[1:]
lista_columnas = df_catusita.columns.tolist()
for hoja in list_hojas:
    df_catusita_hoja = pd.read_excel(file_path, sheet_name=hoja, header=None)
    df_catusita_hoja.columns = lista_columnas
    df_catusita = pd.concat([df_catusita, df_catusita_hoja], ignore_index=True)

KeyboardInterrupt: 

In [14]:
#df_catusita[df_catusita['Fuente de Suministro']=='Deruibo']
df_catusita[df_catusita['Fuente de Suministro']=='DERUIBO']['Artículo'].unique()

array(['KM9.5R17.5', 'DPM12.00R20', 'DP11R22.5', 'DRM12.00R20',
       'KM425/65R22.5', 'DD295/80R22.5', 'DM11R22.5', 'DPP12R22.5',
       'DM12R22.5', 'DRP12.00R24', 'DPP11R22.5', 'LMP235/75R17.5',
       'DM900R20', 'LMD235/75R17.5', 'LMP12R22.5', 'DM12.00R24',
       'SHT265/70R17', 'DM7.50R16LT', 'DP12.00R20', 'DP295/80R22.5',
       'MP11R22.5', 'HD295/80R22.5', 'HM295/80R22.5', 'HD315/80R22.5',
       'HMTLT235/85R16', 'HM315/80R22.5', 'HM11R22.5', 'DM8.25R16LT',
       'LMD215/75R17.5', 'LMP215/75R17.5', 'HD11R22.5', 'DD315/80R22.5',
       'DD11R22.5', 'SD295/80R22.5', 'SM315/80R22.5', 'SM11R22.5',
       'DOP12.00R20', 'DOP11R22.5', 'SD315/80R22.5', 'DPO12.00R20',
       'HD+295/80R22.5', 'KM7.50R16LT', 'KM7.00R16LT', 'LMP7.50R16',
       'CPM8.25R16', 'STM11R22.5', 'STPP12R22.5', 'STP12R22.5',
       'STD295/80R22.5', 'HP295/80R22.5', 'CD215/75R17.5',
       'KD235/75R17.5', 'CP235/75R17.5', 'HDD315/80R22.5', 'PT12.00R20',
       'LM425/65R22.5', 'HDD11R22.5', 'STM12R22.5', '

In [17]:
df_catusita_clean = df_catusita[['Fecha', 'Artículo', 'Venta S/.']].copy()
df_catusita_clean = df_catusita_clean.dropna(subset=['Fecha', 'Artículo', 'Venta S/.'])
df_catusita_clean['Fecha'] = pd.to_datetime(df_catusita['Fecha'], errors='coerce')
df_catusita_clean['Año-Mes'] = df_catusita_clean['Fecha'].dt.to_period('M').astype(str)
df_catusita_clean['Artículo'] = df_catusita_clean['Artículo'].str.strip()
df_catusita_clean['Venta S/.'] = pd.to_numeric(df_catusita_clean['Venta S/.'], errors='coerce')
df_pivot = df_catusita_clean.pivot_table(
    index='Año-Mes', 
    columns='Artículo', 
    values='Venta S/.', 
    aggfunc='sum'
)
#df_pivot_no_nan = df_pivot.dropna(how='all', axis=0).dropna(how='all', axis=1)
# no_faltantes = df_pivot.notna().stack()
# no_faltantes_true = no_faltantes[no_faltantes]
# no_faltantes_true
df_pivot
articulos_unicos = df_catusita[df_catusita['Fuente de Suministro'] == 'DERUIBO']['Artículo'].unique()
df_pivot_filtrado = df_pivot.loc[:, df_pivot.columns.isin(articulos_unicos)]
df_pivot_no_nan = df_pivot_filtrado.fillna('na')


In [18]:
df_pivot_no_nan.describe()

Artículo,CD215/75R17.5,CP235/75R17.5,CPM8.25R16,DD11R22.5,DD295/80R22.5,DD315/80R22.5,DM11R22.5,DM12.00R24,DM12R22.5,DM7.50R16LT,...,SD295/80R22.5,SD315/80R22.5,SHT265/70R17,SM11R22.5,SM315/80R22.5,STD295/80R22.5,STM11R22.5,STM12R22.5,STP12R22.5,STPP12R22.5
count,48,48,48,48,48,48,48,48,48,48,...,48,48,48,48,48,48,48,48,48,48
unique,4,3,6,4,18,4,15,16,7,5,...,7,3,5,4,8,3,6,2,4,3
top,na,na,na,na,na,na,na,na,na,na,...,na,na,na,na,na,na,na,na,na,na
freq,45,46,43,45,31,45,34,33,42,44,...,42,46,44,45,41,46,43,47,45,46


In [19]:
df_pivot_no_nan.to_excel('deruibo_only.xlsx')

In [1]:
import pandas as pd 
from utils.rfm.rfm_processor import process_rfm
from utils.process_data.config import DATA_PATHS
df_catusita = pd.read_csv(DATA_PATHS['process'] / 'catusita_consolidated.csv')
lista_skus_rfm, df_rfm = process_rfm(df_catusita)
df_skus_rfm = pd.DataFrame({'sku': lista_skus_rfm})

RFM SKUs represent 86.58% of total sales


In [12]:
df_rfm[['sku','rfm']].to_csv('data/process/df_rfm.csv')

In [1]:
from utils.dashboard.dashboard import DataProcessor
from utils.process_data.config import DATA_PATHS
base_path = str(DATA_PATHS['cleaned'].parent.parent)
processor = DataProcessor(base_path)
processor.process_all()
processor.dfdashboard.to_csv(DATA_PATHS['cleaned'] / 'dashboard.csv', index=False)
processor.dffinal3.to_csv(DATA_PATHS['cleaned'] / 'dashboard_by_fuente.csv', index=False)
processor.dffinal2.to_csv(DATA_PATHS['cleaned'] / 'download.csv', index=False)

[*********************100%***********************]  4 of 4 completed
c:\Users\YOGA\Desktop\repositories\caa\catusita\catusita_predictions\utils\dashboard\dashboard.py:68: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.long_format['Closing Price'] = self.long_format.groupby('Currency Pair')['Closing Price'].fillna(method='ffill')


In [8]:
import pandas as pd  
pd.set_option('display.max_rows', None)  # Mostrar todas las filas
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas

# Seleccionar solo las columnas involucradas
columns_to_display = [
    'Artículo', 'Fuente Suministro', 'Inventario', 'Backorder', 
    'Demanda Mensual', 'holgura', 'consumiendo_proteccion', 
    'quebro', 'va_a_quebrar', 'verde', 'amarillo', 'rojo', 'Alerta'
]

# Ver el DataFrame final con las columnas seleccionadas
df_final = processor.dffinal2[columns_to_display]
# df_temp = df_final[(df_final['consumiendo_proteccion']==0) & (df_final['va_a_quebrar']==1)]
# df_temp
df_final[(df_final['consumiendo_proteccion']==1)&(df_final['va_a_quebrar']==1)]

,Artículo,Fuente Suministro,Inventario,Backorder,Demanda Mensual,holgura,consumiendo_proteccion,quebro,va_a_quebrar,verde,amarillo,rojo,Alerta
1,t008,ito koyama - resortes,0.0,0.0,119.852539,0.000000,1,1,1,0,0,1,rojo
2,a-1331-s,sakura filter,353.0,0.0,500.014730,0.705979,1,0,1,0,1,0,amarillo
4,54651-2h000,ito koyama - amortiguadores,0.0,0.0,23.673817,0.000000,1,1,1,0,0,1,rojo
11,cg001,ito koyama - resortes,0.0,0.0,11.740377,0.000000,1,1,1,0,0,1,rojo
20,eo-65090,sakura filter,74.0,0.0,80.000000,0.925000,1,0,1,0,1,0,amarillo
23,48510-09k60ref,ito koyama - amortiguadores,0.0,0.0,74.000000,0.000000,1,1,1,0,0,1,rojo
26,bdd084,ishikawa - discos de freno,0.0,0.0,8.364519,0.000000,1,1,1,0,0,1,rojo
28,c001,ito koyama - resortes,0.0,0.0,35.995519,0.000000,1,1,1,0,0,1,rojo
41,48541-09210ref,ito koyama - amortiguadores,0.0,0.0,29.500000,0.000000,1,1,1,0,0,1,rojo
44,mi004,ito koyama - resortes,0.0,0.0,14.001086,0.000000,1,1,1,0,0,1,rojo
